# Fast Fourier (Welch Method)

In [ ]:
# # For each retained epoch, spectral features were computed using FFT (Welch method) to derive absolute and relative band powers. 

import mne
import numpy as np
import pandas as pd
import os
from mne.time_frequency import psd_array_welch

BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJ = "sub-001"
SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
DATA_DIR = os.path.join(SUBJ_DIR, "data")
EPO_FILE = os.path.join(SUBJ_DIR, "epo_001_raw.fif")

epochs = mne.read_epochs(EPO_FILE, preload=True)
sfreq = epochs.info["sfreq"]
ch_names = epochs.ch_names
data = epochs.get_data()

BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha":(8,13),
    "alpha1": (8, 10.5),
    "alpha2": (10.5,13),
    "beta": (13, 30),
    "high_beta_low_gamma": (30, 45) # Will not rely on much

    
}

# # --- Compute PSD ---
# n_per_seg = int(sfreq * 4)  # 4s windows
# psds, freqs = psd_array_welch(
#     data, sfreq=sfreq, fmin=1, fmax=45,
#     n_fft=512, n_per_seg=n_per_seg, window='hann', average='mean'
# )
# df = freqs[1] - freqs[0]

# # --- Extract band powers ---
# band_results = []
# for b_name, (fmin, fmax) in BANDS.items():
#     idx = np.logical_and(freqs >= fmin, freqs <= fmax)
#     abs_power = psds[:, :, idx].sum(axis=-1) * df
#     total_power = psds.sum(axis=-1) * df
#     rel_power = abs_power / total_power
#     band_results.append({'band': b_name, 'abs': abs_power, 'rel': rel_power})

# # --- Save CSV per epoch/channel ---
# os.makedirs(DATA_DIR, exist_ok=True)
# rows = []
# for e_idx in range(len(data)):
#     for c_idx, ch in enumerate(ch_names):
#         entry = {'epoch': e_idx, 'channel': ch}
#         for res in band_results:
#             entry[f"{res['band']}_abs"] = res['abs'][e_idx, c_idx]
#             entry[f"{res['band']}_rel"] = res['rel'][e_idx, c_idx]
#         rows.append(entry)
# df_epochs = pd.DataFrame(rows)
# df_epochs.to_csv(os.path.join(DATA_DIR, f"{SUBJ}_bandpowers_epoch.csv"), index=False)


In [ ]:
SUBJECTS = [f"sub-{i:03d}" for i in range(46,150)]  # adjust range

for SUBJ in SUBJECTS:
    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
    DATA_DIR = os.path.join(SUBJ_DIR, "data")
    EPO_FILE = os.path.join(SUBJ_DIR, f"epo_{SUBJ[-3:]}_raw.fif")
    
    epochs = mne.read_epochs(EPO_FILE, preload=True)
    sfreq = epochs.info["sfreq"]
    ch_names = epochs.ch_names
    data = epochs.get_data()

    BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha1": (8, 10.5),
    "alpha2": (10.5,13),
    "beta": (13, 30),
    "high_beta_low_gamma": (30, 45) # Will not rely on much

    }

    # --- Compute PSD ---
    n_per_seg = int(sfreq * 4)  # 4s windows
    psds, freqs = psd_array_welch(
        data, sfreq=sfreq, fmin=1, fmax=45,
        n_fft=512, n_per_seg=n_per_seg, window='hann', average='mean'
    )
    df = freqs[1] - freqs[0]

    # --- Extract band powers ---
    band_results = []
    for b_name, (fmin, fmax) in BANDS.items():
        idx = np.logical_and(freqs >= fmin, freqs <= fmax)
        abs_power = psds[:, :, idx].sum(axis=-1) * df
        total_power = psds.sum(axis=-1) * df
        rel_power = abs_power / total_power
        band_results.append({'band': b_name, 'abs': abs_power, 'rel': rel_power})

    # --- Save CSV per epoch/channel ---
    os.makedirs(DATA_DIR, exist_ok=True)
    rows = []
    for e_idx in range(len(data)):
        for c_idx, ch in enumerate(ch_names):
            entry = {'epoch': e_idx, 'channel': ch}
            for res in band_results:
                entry[f"{res['band']}_abs"] = res['abs'][e_idx, c_idx]
                entry[f"{res['band']}_rel"] = res['rel'][e_idx, c_idx]
            rows.append(entry)
    df_epochs = pd.DataFrame(rows)
    df_epochs.to_csv(os.path.join(DATA_DIR, f"{SUBJ}_bandpowers_epoch.csv"), index=False)
        

In [ ]:
print(epochs.info['sfreq'])


In [ ]:

SUBJECTS = [f"sub-{i:03d}" for i in range(1,45)]  # adjust range

for SUBJ in SUBJECTS:
    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ)
    DATA_DIR = os.path.join(SUBJ_DIR, "data")
    EPO_FILE = os.path.join(SUBJ_DIR, f"epo_{SUBJ[-3:]}_raw.fif")
    
    epochs = mne.read_epochs(EPO_FILE, preload=True)
    sfreq = epochs.info["sfreq"]
    ch_names = epochs.ch_names
    data = epochs.get_data()

    # Only alpha band
    ALPHA_BAND = (8, 13)

    # --- Compute PSD ---
    n_per_seg = int(sfreq * 4)  # 4s windows
    psds, freqs = psd_array_welch(
        data, sfreq=sfreq, fmin=1, fmax=45,
        n_fft=512, n_per_seg=n_per_seg, window='hann', average='mean'
    )
    df = freqs[1] - freqs[0]

    # --- Extract alpha power ---
    idx = np.logical_and(freqs >= ALPHA_BAND[0], freqs <= ALPHA_BAND[1])
    abs_alpha = psds[:, :, idx].sum(axis=-1) * df
    total_power = psds.sum(axis=-1) * df
    rel_alpha = abs_alpha / total_power

    # --- Save CSV per epoch/channel ---
    os.makedirs(DATA_DIR, exist_ok=True)
    rows = []
    for e_idx in range(len(data)):
        for c_idx, ch in enumerate(ch_names):
            entry = {
                'epoch': e_idx,
                'channel': ch,
                'alpha_abs': abs_alpha[e_idx, c_idx],
                'alpha_rel': rel_alpha[e_idx, c_idx]
            }
            rows.append(entry)
    df_epochs = pd.DataFrame(rows)
    df_epochs.to_csv(os.path.join(DATA_DIR, f"{SUBJ}_alpha_band_epoch.csv"), index=False)
